<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-принятия-решений:" data-toc-modified-id="Дерево-принятия-решений:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево принятия решений:</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Проверка-на-тестовой-выборке" data-toc-modified-id="Проверка-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка на тестовой выборке</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Обзор данных

Начнем исследование с общего обзора данных. Для этого импортируем библиотеку `pandas`, прочитаем с её помощью файлы с данными и оценим формат таблицы с помощью метода `head()`. Также сразу импортируем другие инструменты, которые пригодятся нам в процессе исследования. 

In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Прочитаем файл и сохраним его в переменной df для дальнейшей работы (используем конструкцию try-except, т.к. проект выполнялся локально, а не на платформе):

In [2]:
try: 
    df = pd.read_csv('C:/Users/freak/Desktop/Python/ML_introduction_project/users_behavior.csv')
    
except FileNotFoundError:
    df = pd.read_csv('/datasets/users_behavior.csv')

Ознакомимся с внешним видом таблицы:

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Столбцы названы корректно, проблем с регистром нет. 

Проверим типы данных: 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Здесь сразу можно увидеть отсутствие пропусков в данных. 

Количество звонков и количество сообщений можно привести к целочисленному типу:

In [5]:
df['calls'] = df['calls'].astype('int')

In [6]:
df['messages'] = df['messages'].astype('int')

Проверим на дубликаты:

In [7]:
df.duplicated().sum()

0

**Вывод:**

Качество данных хорошее. Пропусков, дубликатов и других проблем не обнаружено. 

## Разделение на выборки

Для начала выделим признаки и целевой признак из исходного датасета:

In [8]:
features = df.drop(['is_ultra'], axis=1)

In [9]:
target = df['is_ultra']

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. 

In [10]:
df_train, df_valid, df_test = np.split(df, [int(.7*len(df)), int(.85*len(df))])

Объявим переменные с признаками для обучения: 

In [11]:
#для обучающей выборки
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

#для валидационной выборки
features_valid = df_valid.drop(['is_ultra'], axis=1)  
target_valid = df_valid['is_ultra']

#для тестовой выборки
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Выборки готовы, можно переходить к исследованию моделей. 

## Исследование моделей

Для бинарной классификации будем использовать следующие модели: 
    
- дерево принятия решений (Decision Tree Classifier)

- случайный лес (Random Forest Classifier)

- логистическая регрессия (Logistic Regression)

### Дерево принятия решений:

In [12]:
model_tree = DecisionTreeClassifier(random_state=12345)
model_tree.fit(features_train, target_train)
predictions_valid = model_tree.predict(features_valid)
accuracy_tree = accuracy_score(target_valid, predictions_valid)

print('Accuracy для дерева решений:', round(accuracy_tree, 2))

Accuracy для дерева решений: 0.73


Теперь изменим проверим, как будет работать модель с разной глубиной деревьев. Изменим гиперпараметр max depth, возьмем для него значения от 1 до 10:

In [13]:
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 

    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)

    print("max_depth =", depth, ": ", end='')
    print(round(accuracy_score(target_valid, predictions_valid), 2))


max_depth = 1 : 0.76
max_depth = 2 : 0.78
max_depth = 3 : 0.8
max_depth = 4 : 0.8
max_depth = 5 : 0.8
max_depth = 6 : 0.82
max_depth = 7 : 0.81
max_depth = 8 : 0.8
max_depth = 9 : 0.81
max_depth = 10 : 0.81


Наилучший результат составляет 0.82 и достигается при максимальной глубине 6. Такой результат соответствует техническому заданию.

### Случайный лес

In [14]:
model_forest = RandomForestClassifier(random_state=12345) 
model_forest.fit(features_train, target_train)  

predictions_forest = model_forest.predict(features_valid) 
accuracy_forest = accuracy_score(target_valid, predictions_forest)

print('Accuracy для случайного леса:', round(accuracy_forest, 2))

Accuracy для случайного леса: 0.8


Результат работы случайного леса заметно выше, чем у дерева решений. Посмотрим, удастся ли его улучшить с помощью изменения гиперпараметров. 

In [15]:
best_model = None
best_result = 0
for depth in range(1, 11):
    for est in range(10, 51, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model = model 
            best_result = result

print("Гиперпараметры наилучшей модели на валидационной выборке:", best_model)
print("Accuracy наилучшей модели на валидационной выборке:", best_result)


Гиперпараметры наилучшей модели на валидационной выборке: RandomForestClassifier(max_depth=9, n_estimators=10, random_state=12345)
Accuracy наилучшей модели на валидационной выборке: 0.8298755186721992


Итак, выше можно увидеть, что при глубине дерева 9 и количестве деревьев 10 качество модели увеличилось с 0.8 до почти 0.83

### Логистическая регрессия

In [16]:
model_logistic_reg = LogisticRegression(random_state=12345)
model_logistic_reg.fit(features_train, target_train)
model_logistic_reg.predict(features_valid)
model_logistic_reg.score(features_valid, target_valid)


0.7531120331950207

Логистическая регрессия дает результат лучше, чем дерево решений, но заметно хуже, чем случайный лес.

## Проверка на тестовой выборке

In [17]:
test_DT = model_tree.predict(features_test) #дерево решений
test_RF = model_forest.predict(features_test) #случайный лес
test_LR = model_logistic_reg.predict(features_test) #логистическая регрессия

In [18]:
accuracy_DT_test = accuracy_score(test_DT, target_test) #дерево решений
accuracy_RF_test = accuracy_score(test_RF, target_test) #случайный лес
accuracy_LR_test = accuracy_score(test_LR, target_test) #логистическая регрессия

In [19]:
print('Дерево решений:', accuracy_DT_test)
print('Случайный лес:', accuracy_RF_test)
print('Логистическая регрессия:', accuracy_LR_test)

Дерево решений: 0.7308488612836439
Случайный лес: 0.8240165631469979
Логистическая регрессия: 0.7246376811594203


Наилучший и при этом единственный соответствующий техническому заданию результат показал Случайный лес. 

## Вывод

Самые высокие значения accuracy на валидационной и тестовой выборках показал Случайный лес. 